In [ ]:
import gymnasium as gym
from reward import refined_pnl
from preprocess import preprocess

gym.register(
    id="MultiDatasetDiscretedTradingEnv",
    entry_point="manual_env:MultiDatasetDiscretedTradingEnv",
    disable_env_checker=True,
)

In [ ]:
env_cfg = dict(
    id="MultiDatasetDiscretedTradingEnv",
    dataset_dir="./15m/*.pkl",
    preprocess=preprocess,
    reward_function=refined_pnl,
    positions=[-3, 0, 3],
    trading_fees=0.0001,
    borrow_interest_rate=0.0003,
    portfolio_initial_value=100,
    max_episode_duration="max",  # 24 * 60,
    verbose=1,
    window_size=120,
    initial_position=0,
)
env = gym.make(**env_cfg)

In [ ]:
"""
{'PNL': 0,
 'ROE': 0,
 'data_RSI_6': 65.32,
 'data_close': 7818.0,
 'data_date_close': Timestamp('2018-07-31 17:07:00'),
 'data_high': 7823.94,
 'data_low': 7815.0,
 'data_open': 7820.0,
 'data_volume': 44.05,
 'date': np.datetime64('2018-07-31T17:06:00.000000000'),
 'entry_price': np.float64(7818.0),
 'entry_valuation': 100.0,
 'idx': 119,
 'last_position_changed_idx': 0,
 'portfolio_distribution_asset': 0,
 'portfolio_distribution_borrowed_asset': 0,
 'portfolio_distribution_borrowed_fiat': 0,
 'portfolio_distribution_fiat': 100.0,
 'portfolio_distribution_interest_asset': 0,
 'portfolio_distribution_interest_fiat': 0,
 'portfolio_valuation': 100.0,
 'position': 0,
 'position_index': 1,
 'prev_position_valuation': 100.0,
 'real_position': 0,
 'reward': 0,
 'step': 0}
"""


In [ ]:
# Run an episode until it ends :
done, truncated = False, False
observation, info = env.reset()

while not done and not truncated:
    ROE = info["ROE"]
    PNL = info["PNL"]
    RSI_6 = info["data_RSI_6"]
    position = info["position"]
    position_index = info["position_index"]
    
    if position == 0:
        if RSI_6 > 75:
            position_index = 0
        elif RSI_6 < 25:
            position_index = 2
    else:
        if ROE > 0.1 or ROE < -0.2: # Take Profit # Stop Loss
            position_index = 1
        else:
            # position_index = 1
            pass

    observation, reward, done, truncated, info = env.step(position_index)

In [ ]:
env.env.save_for_render(dir = "render_logs")


In [ ]:
def RSI(df, periods=6):
    close_delta = df.close.diff()
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com=periods - 1, adjust=True, min_periods=periods).mean()
    ma_down = down.ewm(com=periods - 1, adjust=True, min_periods=periods).mean()

    _rsi = ma_up / ma_down
    return (100 - (100 / (1 + _rsi))).astype(float).round(2)

In [ ]:
from gym_trading_env.renderer import Renderer
renderer = Renderer(render_logs_dir="render_logs")
# renderer.add_line(name= "RSI_6", function=RSI, line_options ={"width" : 1, "color": "blue"})
renderer.run()